# Simulation

In [ ]:
import csv
import time
import dateutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.stats as ss

import buskit as bk
from buskit import busdata as bdata
from buskit import dashboard as bdash
from buskit import simulator as bsim

from IPython.display import display, clear_output

%pylab inline

## 1.1 Global Functions

## 1.2 Global Parameters

## 1.3 Classes

# global clock?

In [ ]:
self.clock = 0
# - pax at stop
# - pax onboard
# - pax arrival: demand profile, arrival function/behavior
# - pax: poisson or exponential

    def update(self):
        self.clock += 1
        self.pax += self.new_pax()
        pax_at_stop[self.link] = self.pax # update global listing of pax at each stop
        return self.pax

    def proceed(self):
        if pax_at_stop[self.link] < self.pax: # if pax are cleared by a bus
            self.pax = pax_at_stop[self.link] # reverse update
            self.clock = 0

        self.update()

In [ ]:
# determine data source
beta = False

time_coef = 100000 # simulation time is __ times faster than the reality
avg_door_t = 5 # assume opening and closing the door take 5 seconds in total
avg_board_t = 3 # assume each boarding takes 3 sec
avg_alight_t = 2 # assume each alight takes 2 sec

# speed and travel time data are currently artificial
link_vel = 1.5 * np.random.randn(len(stop_pos)) + 7 # make sure the unit is m/sec
#dwell_t = 7 * np.random.randn(len(stop_pos)) + 20 # make sure the unit is sec

# pax distribution
stop_pos_next = np.append(stop_pos, stop_pos[-1])[1:]

pos_mu = stop_pos.mean()
pos_std = stop_pos.std()
pax_norm = ss.norm(loc=pos_mu, scale=pos_std)
pax_perc = np.array([pax_norm.cdf(stop_pos_next[i]) - pax_norm.cdf(stop_pos[i]) for i in range(len(stop_pos))]) ### a temporary measure ###

pax_hr_route = 5000
pax_hr_stop = pax_hr_route * pax_perc
pax_at_stop = np.zeros(len(stop_pos))

In [ ]:
# - speed
# - load
# - headway
# - dwelltime
# - acceleration

    capacity = 60
    seat = 40
    
    def __init__(self, ref, pos=0):
        self.ref = ref # vehicle reference
        self.pos = pos # vehicle location (1-D)
        
        ############### fix this: stop_pos, link, vel, next_stop should not require pre-specify, how? ##############
        
        self.link = np.sum(self.pos >= stop_pos) - 1 # link index starts from 0  ### unified with the formula in Stop Class
        self.vel = link_vel[self.link] # speed at current link
        self.next_stop = stop_pos[self.link + 1] # position of next stop
        self.dwell_t = 0
        self.pax = 0
        self.clock = 0
        self.operate = True
        self.atstop = False
        
        self.log_pos = [self.pos]
        self.log_vel = [self.vel]
        self.log_pax = [0]
        self.log_dwell = [0]

    def terminal(self):
        print("The bus has reached the terminal")
        self.operate = False
        self.vel = 0
        self.pax = 0
        
    def stop(self):
        print("Bus %s is making a stop at %s (position %i)"%(self.ref, stop_name[self.link + 1], self.next_stop))
        self.atstop = True
        self.pax_to_board = pax_at_stop[self.link + 1] # check how many pax at stop
        self.board_t = self.pax * avg_board_t
        self.alight_t = 0 * avg_alight_t  #### TO DEVELOP
        self.dwell_t = avg_door_t + self.alight_t + self.board_t # supposed to dwell for this long
        self.clock += 1

#         self.vel = 0
#         self.pos += self.vel
        self.record()

    def move(self):
        pax_at_stop[self.link + 1] = 0 # clear all pax at stop
        self.log_dwell.append(self.dwell_t)
        # move on!
        self.atstop = False
        self.dwell_t = 0
        self.clock = 0
        self.link += 1
        self.pax = 0 # update pax onboard ###################
        self.record()
        self.vel = link_vel[self.link] # new link speed
        self.next_stop = stop_pos[self.link + 1] # new next stop

    def record(self):
        self.log_pos.append(self.pos)
        self.log_pax.append(self.pax)
        
    def proceed(self):
        if self.operate:
            if self.pos + self.vel >= stop_pos[-1]:
                self.terminal()
            elif self.pos + self.vel >= self.next_stop:  ### this judgement restricts from recording vel as 0 at stop, change to sth else
                self.stop()
                if self.clock >= self.dwell_t:
                    self.move()
            else:
                print("Current position of bus %s: %i"%(self.ref, self.pos))
                self.pos += self.vel
                self.record()
        else:
            print("Bus %s is not operating."%(self.ref))